In [2]:
!pip install transformers sentence_transformers chromadb evaluate bitsandbytes rouge_score

In [3]:
from google.colab import files

# Загрузка файла через интерфейс Colab
uploaded = files.upload()

# После загрузки читаем файл (в примере используется текстовый файл)
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as file:
        document_text = file.read()

# Функция для разбиения текста на чанки
def split_into_chunks(text, chunk_size=200):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Разбиваем документ на чанки (по 200 слов)
chunks = split_into_chunks(document_text, chunk_size=200)
print(f"Количество чанков: {len(chunks)}")

Saving HR-бот.txt to HR-бот (1).txt
Количество чанков: 25


In [4]:
import chromadb
import evaluate
from sentence_transformers import SentenceTransformer

# Инициализация модели SentenceTransformer
embedding_model = SentenceTransformer("deepvk/USER-bge-m3")

# Создание клиента ChromaDB
client = chromadb.Client()

rouge = evaluate.load("rouge")

# Загрузка метрики METEOR
meteor = evaluate.load("meteor")


# Создаем коллекцию в ChromaDB без указания функции для эмбеддингов (мы сами будем генерировать их)
collection = client.create_collection(name="rag_texts")

# Генерируем эмбеддинги для чанков с использованием модели SentenceTransformer
embeddings = embedding_model.encode(chunks)  # chunks - это твои текстовые данные, которые нужно закодировать

# Добавляем чанки в коллекцию с эмбеддингами
chunk_ids = [str(i) for i in range(len(chunks))]
collection.add(documents=chunks, embeddings=embeddings, ids=chunk_ids)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

# Настройка для 4-битного квантования
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       # Можно попробовать 'fp4' или 'nf4' для разных типов квантования
    bnb_4bit_use_double_quant=True,  # Включение double quantization для большей экономии памяти
    bnb_4bit_compute_dtype=torch.bfloat16  # Или torch.float16, если ваша видеокарта поддерживает
)

# Загрузка модели с 4-битным квантованием
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [105]:
def get_prompt(user_query, context):
    return f"""Ты специалист тех поддержки, который отвечает на вопросы пользователей по технической документации.
Отвечай на вопрос только на основе документации и не используй никакие дополнительные источники информации.
После завершения ответа больше ничего не говори.
Отвечай на вопросы развернуто в деловом тоне.
Ответ давай только один раз. Не генерируй примеры кода и образцов каких либо документов, если таковых нет в документации.
Не упоминай, что ты отвечаешь по документации или какому-либо контексту.
Не упоминай вопрос пользователя в ответе.
Если ответа нет в документации, напиши: "Я не знаю".

Ответ на вопрос, после генерации которого сразу же нужно остановить генерацию:
{context}

Вопрос пользователя:
{user_query}"""

In [107]:
generation_config = GenerationConfig(
    bos_token_id=128000,         # начальный токен, c которого начинается генерация
    eos_token_id=128009,         # токен остановки генерации
    pad_token_id=128000,         # id паддинга(одинаковая длина последовательностей)
    do_sample=True,             # Используем семплирование
    max_new_tokens=320,          # Ограничиваем количество новых токенов
    repetition_penalty=1,      # Штраф за повторения
    temperature=0.1,             # Степень случайности
    top_k=2,                    # Сужаем выбор токенов до k
    top_p=0.1                   # Ограничиваем вероятность выбора
)
print(generation_config)

def similarity_search(collection, user_query, n_results=4):
    query_embedding = embedding_model.encode([user_query])
    results = collection.query(query_embeddings=query_embedding, n_results=n_results)
    return results['documents'][0]

def rag_with_chunks(query):
    # Ищем релевантные чанки
    retrieved_chunk = similarity_search(collection, query)

    # Создаем промпт с найденным контекстом
    prompt = get_prompt(query, retrieved_chunk)

    # Генерация ответа с помощью модели
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]  # Убираем токены промпта
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return output

references = """Деловой квартал

1.Въезд/выезд на территорию возможен:
а) со стороны Сибирского тракта – с 07.00 до 22.00 открыт въезд и выезд,
б) с переулка Базовый – в утреннее время: с 7:00 до 10:00 открыт въезд, в вечернее время: с 17:00 до 18:30 открыт выезд ,
в) с ул. Ткачей – также в утреннее время: с 7:00 до 10:00 открыт въезд, в вечернее время: с 17:00 до 18:30 открыт выезд.
2. Парковка: у каждого здания имеется бесплатная прилегающая парковка.
На территории также возможна ночная парковка (предварительно предупредив офис-менеджера): автомобиль можно оставить на гостевой парковке, которая расположена через дорогу напротив Строения 2. Оплата стоянки производится через терминал «TelePay», установленный на гостевой парковке и в Строении 2 (1 этаж), или через онлайн сервис «Фрисби». Стоимость – 100 руб./сутки.

"""


# Пример запроса
query = "Как оформить парковку в Деловом квартале?"
answer = rag_with_chunks(query)


rouge_result = rouge.compute(predictions=[answer], references=[references])
print("ROUGE:", rouge_result)

# Подсчет METEOR
meteor_result = meteor.compute(predictions=[answer], references=[references])
print("METEOR:", meteor_result)

print(f"Ответ: {answer}")


GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 320,
  "pad_token_id": 128000,
  "temperature": 0.1,
  "top_k": 2,
  "top_p": 0.1
}

ROUGE: {'rouge1': 0.23529411764705882, 'rouge2': 0.1875, 'rougeL': 0.23529411764705882, 'rougeLsum': 0.23529411764705882}
METEOR: {'meteor': 0.3537705371550652}
Ответ: Ответ:
Парковка в Деловом квартале находится по адресу ул. Ткачей, 21. Въезд/выезд на территорию возможен круглосуточно. Для того, чтобы заехать, можно воспользоваться картой или гостевым талоном, который позднее можно обнулить у офис-менеджера. Ткачей 6. Парковка бесплатная. Оплата стоянки производится через терминал «TelePay», установленный на гостевой парковке и в Строении 2 (1 этаж), или через онлайн сервис «Фрисби». Стоимость – 100 руб./сутки. Шлагбаум находится по левой стороне здания (если стоять лицом к Зданию). Необходимо позвонить по указанному номеру охране и сообщить, что Вы в УЦСБ. Далее нужно объехать здание, Вы с